In [2]:
import os

In [3]:

%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMLProjectGenderClassification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMLProjectGenderClassification'

In [26]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    train_data_arr_path:Path
    model_name:str 
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    random_state: int
    target_column:str

In [27]:
from EndToEndMLProjectGenderClassification.constants import *
from EndToEndMLProjectGenderClassification.utils.common import read_yaml,create_directories

In [28]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH) -> None:
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])      


    def get_training_config(self)-> TrainingConfig:
        config=self.config.training
        params=self.params.RandomForestClassifier
        schema=self.schema
      

        create_directories([config.root_dir])

        training_config = TrainingConfig(
            root_dir=config.root_dir,
            train_data_arr_path=config.train_data_arr_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            min_samples_split=params.min_samples_split,
            min_samples_leaf=params.min_samples_leaf,
            random_state=params.random_state,
            target_column=schema.TARGET_COLUMN

        )

        return training_config

In [29]:
import os
import urllib.request as request
from sklearn.ensemble import RandomForestClassifier
import pickle,joblib
import pandas as pd
import numpy as np
from EndToEndMLProjectGenderClassification import logger

In [30]:

class Training:
    def __init__(self,config:TrainingConfig):
        self.config= config

    def initiate_Training(self):

        with open(self.config.train_data_arr_path, 'rb') as f:
            train_data = np.load(f)  


        x_train,y_train=(
            train_data[:,:-1],train_data[:,-1]
        )

        model=RandomForestClassifier(
            n_estimators= self.config.n_estimators,
            min_samples_split=self.config.min_samples_split,
            min_samples_leaf=self.config.min_samples_leaf,
            random_state=self.config.random_state
            )
        
        model.fit(x_train,y_train)  

        joblib.dump(model,os.path.join(self.config.root_dir,self.config.model_name))      



In [31]:

try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.initiate_Training()
except Exception as e:
    raise e

[2024-09-10 00:49:31,155: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-10 00:49:31,161: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-10 00:49:31,166: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-10 00:49:31,167: INFO: common: created directory at: artifacts]
[2024-09-10 00:49:31,169: INFO: common: created directory at: artifacts/training]
